In [ ]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["MCA_WT"]

# Define the collections
products_collection = db["products"]
customers_collection = db["customers"]
orders_collection = db["orders"]
invoices_collection = db["invoices"]

# Drop collections if they exist for a clean start
products_collection.drop()
customers_collection.drop()
orders_collection.drop()
invoices_collection.drop()

# Sample Data for Products
products_data = [
    {"_id": 1, "name": "Laptop", "price": 60000, "stock": 20},
    {"_id": 2, "name": "Smartphone", "price": 20000, "stock": 50},
    {"_id": 3, "name": "Tablet", "price": 15000, "stock": 30},
    {"_id": 4, "name": "Headphones", "price": 3000, "stock": 100},
    {"_id": 5, "name": "Monitor", "price": 12000, "stock": 15}
]

# Sample Data for Customers
customers_data = [
    {"_id": 1, "name": "Mr. Rajiv", "contact": "rajiv@example.com"},
    {"_id": 2, "name": "Ms. Anita", "contact": "anita@example.com"},
    {"_id": 3, "name": "Mr. Sunil", "contact": "sunil@example.com"},
    {"_id": 4, "name": "Ms. Priya", "contact": "priya@example.com"},
    {"_id": 5, "name": "Mr. Rahul", "contact": "rahul@example.com"}
]

# Sample Data for Orders
orders_data = [
    {"_id": 1, "customer_id": 1, "products": [{"product_id": 1, "quantity": 1}], "total": 60000, "status": "processed"},
    {"_id": 2, "customer_id": 2, "products": [{"product_id": 2, "quantity": 2}], "total": 40000, "status": "processed"},
    {"_id": 3, "customer_id": 1, "products": [{"product_id": 4, "quantity": 5}], "total": 15000, "status": "pending"},
    {"_id": 4, "customer_id": 3, "products": [{"product_id": 3, "quantity": 2}], "total": 30000, "status": "processed"},
    {"_id": 5, "customer_id": 5, "products": [{"product_id": 5, "quantity": 1}], "total": 12000, "status": "pending"}
]

# Sample Data for Invoices
invoices_data = [
    {"_id": 1, "order_id": 1, "invoice_date": "2024-10-01", "amount": 60000},
    {"_id": 2, "order_id": 2, "invoice_date": "2024-10-02", "amount": 40000},
    {"_id": 3, "order_id": 4, "invoice_date": "2024-10-03", "amount": 30000}
]

# Insert Data into Collections
products_collection.insert_many(products_data)
customers_collection.insert_many(customers_data)
orders_collection.insert_many(orders_data)
invoices_collection.insert_many(invoices_data)


InsertManyResult([1, 2, 3], acknowledged=True)

In [3]:
# Query (a): List all products in the inventory.

# Retrieve all products
products = list(products_collection.find({}, {"_id": 0}))  # Exclude the '_id' field for simplicity
for product in products:
    print(product)

{'name': 'Laptop', 'price': 60000, 'stock': 20}
{'name': 'Smartphone', 'price': 20000, 'stock': 50}
{'name': 'Tablet', 'price': 15000, 'stock': 30}
{'name': 'Headphones', 'price': 3000, 'stock': 100}
{'name': 'Monitor', 'price': 12000, 'stock': 15}


In [4]:
# Query (b): List the details of orders with a value > 20000.

# Retrieve orders with total greater than 20000
high_value_orders = list(orders_collection.find({"total": {"$gt": 20000}}, {"_id": 0}))
for order in high_value_orders:
    print(order)

{'customer_id': 1, 'products': [{'product_id': 1, 'quantity': 1}], 'total': 60000, 'status': 'processed'}
{'customer_id': 2, 'products': [{'product_id': 2, 'quantity': 2}], 'total': 40000, 'status': 'processed'}
{'customer_id': 3, 'products': [{'product_id': 3, 'quantity': 2}], 'total': 30000, 'status': 'processed'}


In [5]:
# Query (c): List all the orders which have not been processed (invoice not generated).

# Retrieve all pending orders where status is "pending"
pending_orders = list(orders_collection.find({"status": "pending"}, {"_id": 0}))
for order in pending_orders:
    print(order)

{'customer_id': 1, 'products': [{'product_id': 4, 'quantity': 5}], 'total': 15000, 'status': 'pending'}
{'customer_id': 5, 'products': [{'product_id': 5, 'quantity': 1}], 'total': 12000, 'status': 'pending'}


In [9]:
# Query (d): List all orders along with their invoices for “Mr. Rajiv”.

# Get the customer ID for "Mr. Rajiv"
customer = customers_collection.find_one({"name": "Mr. Rajiv"}, {"_id": 1})
if customer:
    cid = customer["_id"]
    
    # Find all orders by this customer
    rajiv_orders = list(orders_collection.find({"customer_id": cid}, {"_id": 1, "status": 1}))
    
    for order in rajiv_orders:
        # Check if there's an invoice for each order
        print("order : ",order)
        if order['status'] == 'processed':
            invoice = invoices_collection.find_one({"order_id": order["_id"]}, {"_id": 0})
            
            print( "\tinvoice : ", invoice)

order :  {'_id': 1, 'status': 'processed'}
	invoice :  {'order_id': 1, 'invoice_date': '2024-10-01', 'amount': 60000}
order :  {'_id': 3, 'status': 'pending'}
